In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from src.metrics import *
from settings import *
from src.data import generate

instruments = ['Guitar', 'Accordion', 'Violin', 'Piano']

datasets = [(pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED, f'data_{instrument.lower()}.pkl')), instrument) for instrument in instruments]

2023-05-16 08:51:26.459457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 08:51:26.568407: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 08:51:27.164305: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-05-16 08:51:27.164397: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

In [2]:
import random
import keras
import os, glob
import logging
import librosa, librosa.display

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.layers import Activation, Dense, Dropout, Conv2D, \
                         Flatten, MaxPooling2D
from keras.models import Sequential, model_from_json
from keras import backend as K

from src.metrics import *
from settings import *
from src.data import generate
from src.processing import *
from src.model import CNN
from src.data.preprocessing import get_most_shape
from setup_logging import setup_logging

setup_logging()
logger = logging.getLogger('src.train')

augmented = False

train_datas = []
test_datas = []
test_instruments = []
for dataset, insrument in datasets:
    train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.8)
    train_datas.append(train_data)
    test_datas.append(test_data)

    test_instruments.append(insrument)

train_data = pd.concat(train_datas)
test_data = pd.concat(test_datas)

most_shape = get_most_shape(train_data)

logger.info(f"Number of train samples: {len(train_data)}")
logger.info(f"Number of test samples: {len(test_data)}")
# most_shape = get_most_shape(dataset)
#train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.65)

X_train, y_train = features_target_split(train_data)
X_test, y_test = features_target_split(test_data)

# Reshape for CNN input
X_train, X_test = reshape_feature_CNN(X_train), reshape_feature_CNN(X_test)

# Preserve y_test values
y_test_values = y_test.copy()

# One-Hot encoding for classes
y_train, y_test = one_hot_encode(y_train), one_hot_encode(y_test)

# Instance of CNN model
cnn = CNN(most_shape)
logger.info(str(cnn))

cnn.train(X_train, y_train, X_test, y_test)
#cnn.evaluate(X_train, y_train, X_test, y_test)

if tf.__version__ != '1.8.0':
    predict_x=cnn.model.predict(X_test)
    predictions = np.argmax(predict_x,axis=1)
else:    
    predictions = cnn.model.predict_classes(X_test)

#conf_matrix=confusion_matrix(y_test_values, predictions, labels=range(10))
#logger.info('Confusion Matrix for classes {}:\n{}'.format(CLASSES, conf_matrix))

src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 1600
src.processing - INFO - Number of testing samples is 400
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 80
src.processing - INFO - Number of testing samples is 20
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 80
src.processing - INFO - Number of testing samples is 20
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.8
src.processing - INFO - Number of training samples is 80
src.processing - INFO - Number of testing samples is 20
src.processing - INFO - Train test split completed
The most frequent shape is (128, 87)
src.train - INFO - Number of

2023-05-16 08:51:32.941476: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22324 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:3b:00.0, compute capability: 8.6


Epoch 1/67


2023-05-16 08:51:36.086723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2023-05-16 08:51:36.890369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


92/92 [==============================] - 6s 15ms/step - loss: 2.7450 - accuracy: 0.1022 - precision: 0.0815 - recall: 0.0076 - fmeasure: 0.0136 - val_loss: 2.2937 - val_accuracy: 0.1043 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_fmeasure: 0.0000e+00
Epoch 2/67
92/92 [==============================] - 1s 12ms/step - loss: 2.2815 - accuracy: 0.1370 - precision: 0.0652 - recall: 0.0038 - fmeasure: 0.0072 - val_loss: 2.2713 - val_accuracy: 0.1326 - val_precision: 0.0942 - val_recall: 0.0109 - val_fmeasure: 0.0194
Epoch 3/67
92/92 [==============================] - 1s 12ms/step - loss: 2.2393 - accuracy: 0.1641 - precision: 0.2627 - recall: 0.0174 - fmeasure: 0.0323 - val_loss: 2.2140 - val_accuracy: 0.1826 - val_precision: 0.1565 - val_recall: 0.0152 - val_fmeasure: 0.0266
Epoch 4/67
92/92 [==============================] - 1s 12ms/step - loss: 2.2352 - accuracy: 0.1685 - precision: 0.3116 - recall: 0.0212 - fmeasure: 0.0392 - val_loss: 2.1900 - val_accuracy: 0.1674 - val_p

In [4]:
for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, 87, 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = cnn.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])

13/13 [==============================] - 0s 9ms/step - loss: 0.4443 - accuracy: 0.9100 - precision: 0.9201 - recall: 0.9087 - fmeasure: 0.9142
Test score for instrument: Guitar
	Test loss: 0.44427064061164856
	Test accuracy: 0.9100000262260437
	Test precision: 0.9200527667999268
	Test recall: 0.9086538553237915
	Test f1-score: 0.9142233729362488
1/1 [==============================] - 0s 28ms/step - loss: 1.4031 - accuracy: 0.7000 - precision: 0.7647 - recall: 0.6500 - fmeasure: 0.7027
Test score for instrument: Accordion
	Test loss: 1.4031049013137817
	Test accuracy: 0.699999988079071
	Test precision: 0.7647058963775635
	Test recall: 0.6499999761581421
	Test f1-score: 0.7027026414871216
1/1 [==============================] - 0s 28ms/step - loss: 0.8092 - accuracy: 0.6500 - precision: 0.6842 - recall: 0.6500 - fmeasure: 0.6667
Test score for instrument: Violin
	Test loss: 0.8092098236083984
	Test accuracy: 0.6499999761581421
	Test precision: 0.6842105388641357
	Test recall: 0.6499999761

In [ ]:
cnn.save_model(name="model_all_data_noaugment_3")